## Longest Common Subsequence Problem

The LCS problem is finding the longest subsequence present in given two sequences in the same order.

In other words, find the longest sequence which can be obtained from the first original sequence by deleting some items and from the second original sequence by deleting other items.

This problem differs from the problem of finding the longest common substring. Unlike substrings, subsequences are not required to occupy consecutive positions within the original string.

For example, consider two sequences, X and Y:
    
    X: ABCBDAB
    Y: BDCABA
    
    LCS are BDAB, BCAB, and BCBA
    The length of the LCS is 4
    
**This problem has optimal substructure.**
The problem can be broken down into smaller, simple subproblems, which can yet be broken down into simpler subproblems until the problem becomes trivial.

1. Consider two sequences, X and Y, of lengths m and n that both end in the same element.
    To find their LCS, shorten each sequence by removing the last element, find the LCS of the shortened sequences, and append the removed element.
    ```
    LCS(X[1...m], Y[1...n]) = LCS(X[1...m-1], Y[1...n-1]) + X[m] if X[m] = Y[n]
    ```
    
2. Suppose the two sequences do not end in the same element.
    Then the LCS of X and Y is the longer of the two sequences returned by:
    ```
    LCS(X[1...m-1], Y[1...n]) and LCS(X[1...m], Y[1...n-1])
    ```
    

In [27]:
"""
Without dynamic programming, also returns one of the possible subsequences
"""

def LCS_s(x, y):
    lenx = len(x)
    leny = len(y)
    if lenx == 0 or leny == 0:
        return ("", 0)
    if x[-1] == y[-1]:
        s, t = LCS(x[0:lenx-1], y[0:leny-1])
        s += x[-1]
        t += 1
        return (s, t)
    else:
        l = LCS(x[0:lenx-1], y)
        r = LCS(x, y[0:leny-1])
        return l if l[1] > r[1] else r
        

In [145]:
""" 
Without dynamic programming, does not return possible subsequence
"""
def LCS(x, y):
    lenx = len(x)
    leny = len(y)
    if lenx == 0 or leny == 0:
        return 0
    if x[-1] == y[-1]:
        t = LCS(x[0:lenx-1], y[0:leny-1])
        t += 1
        return t
    else:
        l = LCS(x[0:lenx-1], y)
        r = LCS(x, y[0:leny-1])
        return l if l > r else r
        

In [127]:
X = "ABCBDAB"
Y = "BDCABA"
A = "ABABDABCCCABCACABAB"
B = "BABACBAABBDDAADCCAAB"
C = "XMJYAUZ"
D = "MZJAWXU"
E = "ABABDABCCCABCACA"
F = "BABACBAABBDDAADCC"
G = "ABCBA"
H = "BCACA"

In [146]:
LCS(X, Y)

4

In [147]:
%%time
LCS(A, B)

CPU times: user 3.67 s, sys: 43.6 ms, total: 3.71 s
Wall time: 4.17 s


12

### With dynamic programming

**Bottom up approach**

We start with smaller problems. We start with just looking at the first letters of the sequences. If they match, we know we have a common subsequence. If not, we can add a letter to one of the sequences and see if they match. Keep going until we find a match.

    CBCCB and BBCBB, X* indicates we're looking at that letter
    
    C and B*   no match
    C and BB*  no match
    C and BBC* one match possible, move on
    
    CB* and BBCBB, we look at just the B in CB
    B and B*, that's a match
    B and BB*, we can't reuse that
    B and BBC*, we see that C matched before, so we add that to our current match for 2

So we make our 2D array, think of it as a grid for each letter. Pad it with zeros at the top and left.

As we compare each letter, if a match is found, we have to be sure it wasn't already taken for a previous match. We do this by looking left and looking above. If left and above are equal, we found a new match and can increment. If not, we do not have a new match and we take the larger of the two.

        0 C B C C B
      0 0 0 0 0 0 0
      B 0 0 1 1 1 1
      B 0 0 1 1 1 2
      C 0 1 1 2 2 2
      B 0 1 1 2 2 3
      B 0 1 2 2 2 3

In [124]:
"""
with dynamic programming
bottom up approach
starting with small subproblem and build up
"""
def LCS_dy(x, y):
    lookup = [[0 for _ in range(len(x)+1)] for _ in range(len(y)+1)]
    # letters x[0],y[0] correspond to lookup[1][1]
    # letter x[a], y[b] correspond to lookup[b+1][a+1]
    # letters x[2], y[3] correspond to lookup[4][3]
    # to look one row up and one row to the left: lookup[3][2], lookup[b][a]
    # to look to the row above: lookup[3][3], lookup[b][a+1]
    # to look to the column to the left: lookup[4][2], lookup[b+1][a]
    for b in range(len(y)):
        for a in range(len(x)):
            above = lookup[b][a+1]
            left = lookup[b+1][a]
            upleft = lookup[b][a]
            # use upleft to determine your value
            # this ensures the letter you're looking at isn't already being used as the last letter
            if x[a] == y[b]:
                lookup[b+1][a+1] = upleft + 1
            else:
                lookup[b+1][a+1] = max(above, left)
    """ to see one of the possible sequences """
#     sequence = ""
#     h = len(y)
#     w = len(x)
#     max_number = lookup[h][w]
#     while max_number > 0:
#         while lookup[h-1][w-1] == max_number:
#             h -= 1
#             w -= 1
#         while lookup[h-1][w] == max_number:
#             h -= 1
#         while lookup[h][w-1] == max_number:
#             w -= 1
#         sequence = y[h-1] + sequence
#         h -= 1
#         w -= 1
#         max_number -= 1

#     for arr in lookup:
#         print(f'{arr}\n')
    return lookup[len(y)][len(x)]
                    

In [125]:
%%time
LCS_dy(X, Y)

CPU times: user 40 µs, sys: 0 ns, total: 40 µs
Wall time: 42.9 µs


4

In [126]:
%%time
LCS_dy(A,B)

CPU times: user 230 µs, sys: 0 ns, total: 230 µs
Wall time: 233 µs


12

### With dynamic programming

**Top down approach**

We start with the bigger problem and cut it down into smaller problems.

We can use recursion, but we stop recursion from going down repetitive branches by keeping the results from those branches readily available in a lookup table.

In [149]:
""" top down approach"""

def LCS_dy2(x, y, x_pos=None, y_pos=None, lookup=None):
    if lookup == None:
        lookup = {}
    if x_pos == None:
        x_pos = len(x) - 1
        y_pos = len(y) - 1
    if x_pos < 0 or y_pos < 0:
        return 0
    key = f'{x_pos}-{y_pos}'
    if lookup.get(key) == None:
        if x[x_pos] == y[y_pos]:
            length = LCS_dy2(x, y, x_pos-1, y_pos-1, lookup)
            lookup[key] = length + 1
            return length + 1
        else:
            length = max(LCS_dy2(x, y, x_pos-1, y_pos, lookup), LCS_dy2(x, y, x_pos, y_pos-1, lookup))
            lookup[key] = length
            return length
    else:
        return lookup[key]
     
    

In [150]:
%%time
LCS_dy2(A, B)

CPU times: user 404 µs, sys: 16 µs, total: 420 µs
Wall time: 425 µs


12

In [151]:
LCS_dy2("abc", "def")

0